In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from tqdm import tqdm
import shutil
from torchvision import datasets, transforms

from model import ResNet50, ResNet38, ResNet26



In [2]:

## 데이터 셋은 CIFAR10으로
num_classes = 10
batch_size = 10
num_workers = 4

## 데이터셋 로드
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.4914, 0.4822, 0.4465),
        std=(0.2470, 0.2435, 0.2616)
    )
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.4914, 0.4822, 0.4465),
        std=(0.2470, 0.2435, 0.2616)
    )
])
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data', train=True, download=True, transform=transform_train),
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers
)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data', train=False, transform=transform_test),
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers
)

Files already downloaded and verified


In [3]:
stem = False

model = ResNet26(num_classes=num_classes, stem=stem)
model = model.cuda()

# {'img_size': 32, 'batch_size': 10, 'num_workers': 0, 'epochs': 100, 'print_interval': 100, 'cuda': True,  'gpu_devices': None, 'gpu': None, 'rank': 0, 'world_size': 1, 'dist_backend': 'nccl', 'dist_url': 'tcp://127.0.0.1:3456'}


In [4]:
start_epoch = 1
best_acc = 0.0

lr = 0.1
momentum =0.9
weight_decay = 0.0001


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)


In [5]:
epochs = 100
print_interval = 20

## 30에폭 마다 러닝레이트 줄임
def adjust_learning_rate(optimizer, epoch):
    decayed_lr = lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = decayed_lr

for epoch in range(1,epochs+1):
    model.train()
    train_acc = 0.0
    step = 0
    for data, target in train_loader:
        adjust_learning_rate(optimizer, epoch)
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()


        ## onnx 저장


        y_pred = output.data.max(1)[1]
        
        acc = float(y_pred.eq(target.data).sum()) / len(data) * 100.
        train_acc += acc
        step += 1
        if step % print_interval == 0:
            # print("[Epoch {0:4d}] {1:4d}/{1:4d} Loss: {1:2.3f} Acc: {2:.3f}% \n".format(epoch,len(train_loader),step, loss.data, acc), end='')
            print("Epoch ",epoch," ",len(train_loader)," / ",step,"batch Loss: ",loss.item(),"  ACC: ",acc,"%")

    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            prediction = output.data.max(1)[1]
            correct += prediction.eq(target.data).sum()

    acc = 100. * float(correct) / len(test_loader.dataset)
    print('Test acc: {0:.2f} \n'.format(acc))

Epoch  1   5000  /  20 batch Loss:  17.49565315246582   ACC:  0.0 %
Epoch  1   5000  /  40 batch Loss:  3.066394329071045   ACC:  10.0 %
Epoch  1   5000  /  60 batch Loss:  2.4563772678375244   ACC:  0.0 %
Epoch  1   5000  /  80 batch Loss:  2.42293119430542   ACC:  0.0 %
Epoch  1   5000  /  100 batch Loss:  2.229726552963257   ACC:  30.0 %
Epoch  1   5000  /  120 batch Loss:  2.2393100261688232   ACC:  10.0 %
Epoch  1   5000  /  140 batch Loss:  2.459568977355957   ACC:  10.0 %
Epoch  1   5000  /  160 batch Loss:  2.280266284942627   ACC:  10.0 %
Epoch  1   5000  /  180 batch Loss:  2.539658546447754   ACC:  0.0 %
Epoch  1   5000  /  200 batch Loss:  2.365104913711548   ACC:  10.0 %
Epoch  1   5000  /  220 batch Loss:  2.332296371459961   ACC:  0.0 %
Epoch  1   5000  /  240 batch Loss:  2.2190282344818115   ACC:  30.0 %
Epoch  1   5000  /  260 batch Loss:  2.2361652851104736   ACC:  0.0 %
Epoch  1   5000  /  280 batch Loss:  2.38037371635437   ACC:  20.0 %
Epoch  1   5000  /  300 batc

KeyboardInterrupt: 

In [ ]:
## onnx 저장
## operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK 꼭 해줘야함
## unfold 연산을 
# https://github.com/pytorch/pytorch/issues/14395#issuecomment-444697403


# import torch.onnx
# torch.onnx.export(model.cpu(),data.cpu(),'./stand_alone_self.onnx',export_params=False,opset_version=12,
#                   operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK)

In [9]:
print(model.ch)

TypeError: object of type 'generator' has no len()